# Quality Measure Portal

## special nodes
 - Pass id = 99
 - Fail id = 100
 - Exclude = 101

## Node types
-  hasAValue  : check if there is a value within the given path or not.
-  boolean    : the retrieved value is either True or False.
-  YesNo      : check if there is a Yes or No within the given path.
-  inAList    : check if the fetched item existed in the given list or not.
-  lessThan   : check if the fetched item is less than the given value.
-  LargerThan : check if the fetched item is larger than the given value.
-  inRange    : check if the fetched item existed within a list of two values.


In [48]:
import json

class QMBaseNode:

    # Read node from json and initialize required data
    # Include evaluation method, required paths, expected value(s)
    # and traversal info
    # Override with derived classes as needed.  Especially end states.

    def __init__(self, tree, nodeInfo):
        self.tree = tree

        self.featurePath = nodeInfo['path']

        self.featureType = nodeInfo['featureType']
        self.successValues = nodeInfo['successValues']

        self.YesBranchID = nodeInfo['YesBranchID']
        self.NoBranchID = nodeInfo['NoBranchID']


    # Called by other nodes, or the tree, to evaluate the node and pass
    # execution to the next node
    
#     def evaluate(self, patientData):
#         if self.evalExpression(self.featurePath, patientData) == self.expectedValue:
#             return self.tree.getNode(self.YesBranchID).evaluate(patientData)
#         else:
#             return self.tree.getNode(self.NoBranchID).evaluate(patientData)

    def evaluate(self, patientData):
        
        fetchedValue = self.evalExpression(self.featurePath, patientData)

        # print(f'{self.featurePath} = {fetchedValue}')

        result = False

        if self.featureType == "HasAValue":
            if fetchedValue:
                result = True
            else:
                result = False
        
        
        elif self.featureType == "Boolean":
            result = fetchedValue
        
        
        elif self.featureType == "YesNo":
            if fetchedValue in self.successValues:
                result = True
            else:
                result = False
        
        
        elif self.featureType == "InAList":
            if fetchedValue in self.successValues:
                result = True
            else:
                result = False
        
        
        elif self.featureType == "LessThan":
            if fetchedValue < self.successValues:
                result = True
            else:
                result = False
        
        
        elif self.featureType == "LargerThan":
            if fetchedValue > self.successValues:
                result = True
            else:
                result = False
        
        
        elif self.featureType == "InARange":
            if fetchedValue > self.successValues[0] and fetchedValue <= self.successValues[1]:
                result = True
            else:
                result = False

        
        
        if result:
            return self.tree.getNode(self.YesBranchID).evaluate(patientData)
        
        else:
            return self.tree.getNode(self.NoBranchID).evaluate(patientData)

        
        
    def evalExpression(self, featurePath, patientData):
        path = featurePath.split('.')
        nested_value = patientData
        
        for feature in path:
            if feature in nested_value:
                nested_value = nested_value.get(feature)
            else:
                return None
        return nested_value

In [49]:
class QMEndNode(QMBaseNode):
    def __init__(self, tree, nodeInfo):
        self.tree = tree
        self.featurePath = ""
        self.endState = nodeInfo['result']

    def evaluate(self, patientData=None):
        return str(self.endState)

In [50]:
class QMTree:
    dt_json = None
    def __init__(self):
        self.nodes = dict()

    def readTreeFile(self, path):
        with open(f'{path}.json', 'r') as f:
            self.tree = json.load(f)
            
            
        for item in self.tree['nodes']:
            if item['type']=="base":
                self.nodes[item['featureID']] = QMBaseNode(self, item)
            else:
                self.nodes[item['featureID']] = QMEndNode(self, item)

        
    def addNode(self, node):
        pass
    
    def removeNode(self, node):
        pass
    
    def getNode(self, id):
        try:
            return self.nodes[id]
        except:
            print('Warning: no such node with id ' , id)
            return None
        
    def evaluate(self, startId, patientData):
        return self.getNode(startId).evaluate(patientData)

         
           
    def get_next(self, next_node, dt_json):
        '''search for the next item'''
        print('Next node is : ', next_node)
        for node_dict in dt_json:
            if node_dict['featureID']==next_node:
                return node_dict



In [51]:
myTree = QMTree()
myTree.readTreeFile("DT2")


with open('Data/data.json', 'r') as f:
    dataset = json.load(f)


print([myTree.evaluate("1", patient) for patient in dataset])
# print(myTree.evaluate("1", dataset[0]) )


['fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'pass', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'exclude', 'exclude', 'exclude', 'fail', 'pass', 'pass', 'pass', 'pass', 'pass', 'fail', 'exclude', 'exclude', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'fail', 'fail', 'fail', 'fail', 'fail', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'pass', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 'fail', 

# Types of features
- Task1: extract unique paths

In [52]:
# Showing all unique paths 
def extract_json_paths(json_data):
    paths = []

    def traverse_json(data, path=""):
        if isinstance(data, dict):
            for key, value in data.items():
                new_path = path + "/" + key if path else key
                paths.append({
                    'path': new_path,
                    'type': type(value).__name__,
                    'value': value
                })
                traverse_json(value, new_path)
        elif isinstance(data, list):
            for i, item in enumerate(data):
                new_path = path + "/{}".format(i) if path else str(i)
                paths.append({
                    'path': new_path,
                    'type': type(item).__name__,
                    'value': item
                })
                traverse_json(item, new_path)

    traverse_json(json_data)
    return paths


In [59]:
paths = []
for ds in dataset:
    for e in extract_json_paths(ds):
        paths.append(e)

In [54]:
listPaths = []
strPaths = []
intPaths = []
dictPaths = []

for p in paths:
    if p['type']=='list':
        listPaths.append(p['path'].split('/')[-1])
    elif p['type']=='dict':
        dictPaths.append(p['path'].split('/')[-1])
    elif p['type']=='str':
        strPaths.append(p['path'].split('/')[-1])
    elif p['type']=='int':
        intPaths.append(p['path'].split('/')[-1])
    
listPaths = sorted(set(listPaths))
dictPaths = sorted(set(dictPaths))
intPaths = sorted(set(intPaths))
strPaths = sorted(set(strPaths))
print('list: \n', listPaths)
print('\ndict: \n', dictPaths)
print('\nint : \n', intPaths)
print('\nstr : \n', strPaths)


list: 
 ['adtInjections', 'chemotherapy', 'clinicalTrials', 'consult', 'discussedTreatmentOptions', 'dvhStructuresEvaluated', 'ebrt', 'followUp', 'ldr', 'nsclcMotionManagement', 'otv', 'performanceStatus', 'postTreatmentEvaluation', 'problemList', 'psa', 'qualityOfLife', 'sclcMotionManagement', 'survivorshipCarePlan', 'toxicity', 'treatmentSummary']

dict: 
 ['0', '1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '18', '180', '181', 

In [55]:
strtotalValues = {}
for feature in strPaths:
    strtotalValues[feature] = set([f['value'] for f in paths if f['path'].split('/')[-1]==feature and f['type']=='str'])


In [56]:
for f in strtotalValues.keys():
    if len(strtotalValues[f])<20:
        print(f'{f} : {strtotalValues[f]}\n')

1 : {'ICTV', 'Lung', 'Spinal Cord', 'Bladder', 'Radical Prostatectomy', 'Interstitial Prostate Brachytherapy', 'Follow up care', 'Prostate D90 Eval.', 'Esophagus', 'External Beam RT', 'Rectum', 'Heart', 'Prostate V100 Eval.', 'Relevant assessment of tolerance to and progress towards the treatment goals', 'ADT'}

2 : {'Lung', 'Spinal Cord', 'Bladder', 'Radical Prostatectomy', 'Interstitial Prostate Brachytherapy', 'Follow up care', 'Prostate D90 Eval.', 'Physician Review', 'Esophagus', 'External Beam RT', 'Rectum', 'Heart', 'Rectum V100 Eval.', 'IGTV', 'Relevant assessment of tolerance to and progress towards the treatment goals', 'PTV', 'ADT'}

3 : {'Lung', 'Spinal Cord', 'Radical Prostatectomy', 'Interstitial Prostate Brachytherapy', 'Physician Review', 'Prostate D90 Eval.', 'Esophagus', 'External Beam RT', 'Heart', 'Rectum V100 Eval.', 'ADT'}

4 : {'Rectum V100 Eval.', 'Physician Review', 'CT or MRI'}

adtDuration : {'Long Term', 'Not Stated', 'Short Term'}

boneDensityAssessment : {

In [57]:
intTotalValues = {}
for feature in strPaths:
    temp = [f['value'] for f in paths if f['path'].split('/')[-1]==feature and f['type']=='int']
    if len(temp)>0:
        intTotalValues[feature] = set(temp)


In [58]:
for f in intTotalValues.keys():
    print(f'{f} : {intTotalValues[f]}\n')

performanceScore : {0, 1, 2, 3, 100, 4, 70, 5, 40, 80, 50, 85, 90, 60, 95}

qualityOfLifeScore : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 38, 41, 102}

